In [10]:
from mpa.models import Mpa, mpa_post_save
from django.db.models.signals import post_save, post_delete
from mpa.views import mpas_all_nogeom
import reversion
from reversion.models import Revision
from reversion.models import Version
import datetime
from django.db.models import Q, F, Func

In [11]:
mpaset = mpas_all_nogeom.filter(
    Q(country__icontains='CAN') | Q(sovereign__icontains='CAN')
)
mpaset.values('country').distinct()

<QuerySet [{'country': 'CAN'}]>

In [12]:
mpas_all_nogeom.filter(wdpa_id=0).update(wdpa_id=None)
mpas_all_nogeom.filter(wdpa_id=0).count()

0

In [80]:
count = 0
for m in mpaset:
    versions = Version.objects.get_for_object(m)
    created = 'Never'
    if not m.wdpa_id: continue
    try:
        created = versions.first().revision.date_created
        if created > datetime.datetime(2014, 11, 5):
            count = count + 1
            print(count, created, 'count:', len(versions), 'mpa_id:', m.mpa_id, 'wdpa_id:', m.wdpa_id)
    except:
        pass

1 2018-01-08 09:05:37.717923 count: 7 mpa_id: 7701383 wdpa_id: 18398
2 2018-10-24 14:13:04.022092 count: 1 mpa_id: 68809419 wdpa_id: 555593789
3 2018-10-24 14:13:04.399222 count: 1 mpa_id: 68809420 wdpa_id: 555593790
4 2018-10-24 14:13:04.856898 count: 1 mpa_id: 68809421 wdpa_id: 555593795
5 2018-10-24 14:13:05.246849 count: 1 mpa_id: 68809422 wdpa_id: 555593796
6 2018-10-24 14:13:05.691874 count: 1 mpa_id: 68809423 wdpa_id: 555593797
7 2018-10-24 14:13:06.805409 count: 1 mpa_id: 68809424 wdpa_id: 555593798
8 2018-10-24 14:13:07.183706 count: 1 mpa_id: 68809425 wdpa_id: 555593804
9 2018-10-24 14:13:07.591903 count: 1 mpa_id: 68809426 wdpa_id: 555593811
10 2018-10-24 14:13:08.086732 count: 1 mpa_id: 68809427 wdpa_id: 555593812
11 2018-10-24 14:13:08.489679 count: 1 mpa_id: 68809428 wdpa_id: 555593849
12 2018-10-24 14:13:08.886747 count: 1 mpa_id: 68809429 wdpa_id: 555593850
13 2018-10-24 14:13:11.032451 count: 1 mpa_id: 68809430 wdpa_id: 555593852
14 2018-10-24 14:13:13.387058 count: 1 

145 2018-10-24 14:11:06.353548 count: 1 mpa_id: 68809289 wdpa_id: 555515521
146 2018-10-24 14:11:06.732755 count: 1 mpa_id: 68809290 wdpa_id: 555515525
147 2018-10-24 14:11:07.200196 count: 1 mpa_id: 68809291 wdpa_id: 555515527
148 2018-10-24 14:11:07.584598 count: 1 mpa_id: 68809292 wdpa_id: 555515529
149 2018-10-24 14:11:07.916529 count: 1 mpa_id: 68809293 wdpa_id: 555515530
150 2018-10-24 14:11:08.295904 count: 1 mpa_id: 68809294 wdpa_id: 555515531
151 2018-10-24 14:11:08.713144 count: 1 mpa_id: 68809295 wdpa_id: 555515554
152 2018-10-24 14:11:09.122824 count: 1 mpa_id: 68809296 wdpa_id: 555515555
153 2018-10-24 14:11:09.492459 count: 1 mpa_id: 68809297 wdpa_id: 555515562
154 2018-10-24 14:11:09.829382 count: 1 mpa_id: 68809298 wdpa_id: 555515563
155 2018-10-24 14:11:10.215967 count: 1 mpa_id: 68809299 wdpa_id: 555515566
156 2018-10-24 14:11:10.607567 count: 1 mpa_id: 68809300 wdpa_id: 555515568
157 2018-10-24 14:11:10.974253 count: 1 mpa_id: 68809301 wdpa_id: 555515572
158 2018-10-

278 2018-10-24 14:13:18.519931 count: 1 mpa_id: 68809444 wdpa_id: 555593886
279 2018-10-24 14:13:18.966352 count: 1 mpa_id: 68809445 wdpa_id: 555593887
280 2018-10-24 14:13:19.374965 count: 1 mpa_id: 68809446 wdpa_id: 555593896
281 2018-10-24 14:13:19.791302 count: 1 mpa_id: 68809447 wdpa_id: 555593900
282 2018-10-24 14:13:20.378134 count: 1 mpa_id: 68809448 wdpa_id: 555593901
283 2018-10-24 14:13:20.776028 count: 1 mpa_id: 68809449 wdpa_id: 555621544
284 2018-10-24 14:13:21.231756 count: 1 mpa_id: 68809450 wdpa_id: 555621551
285 2018-10-24 14:13:22.893476 count: 1 mpa_id: 68809451 wdpa_id: 555621553
286 2018-10-24 14:13:23.599526 count: 1 mpa_id: 68809452 wdpa_id: 555621557
287 2018-10-24 14:13:23.936535 count: 1 mpa_id: 68809453 wdpa_id: 555621601
288 2018-10-24 14:13:24.404951 count: 1 mpa_id: 68809454 wdpa_id: 555621668
289 2018-10-24 14:13:24.993626 count: 1 mpa_id: 68809455 wdpa_id: 555621669
290 2018-10-24 14:13:25.452096 count: 1 mpa_id: 68809456 wdpa_id: 555621670
291 2018-10-

405 2014-11-05 16:40:00.801401 count: 1 mpa_id: 4298 wdpa_id: 308442
406 2014-11-05 16:27:25.236165 count: 1 mpa_id: 15246 wdpa_id: 555567634
407 2014-11-05 16:27:25.437763 count: 1 mpa_id: 15252 wdpa_id: 555567643
408 2014-11-05 16:27:25.607712 count: 1 mpa_id: 15257 wdpa_id: 555567657
409 2014-11-05 16:27:25.638835 count: 1 mpa_id: 15258 wdpa_id: 555567659
410 2014-11-05 16:39:52.056069 count: 1 mpa_id: 4102 wdpa_id: 306351
411 2014-11-05 16:27:25.738032 count: 1 mpa_id: 15261 wdpa_id: 555567663
412 2014-11-05 16:27:25.799453 count: 1 mpa_id: 15263 wdpa_id: 555567668
413 2014-11-05 16:27:25.879242 count: 1 mpa_id: 15265 wdpa_id: 555567676
414 2014-11-05 16:27:25.910948 count: 1 mpa_id: 15266 wdpa_id: 555567678
415 2014-11-05 16:27:26.057080 count: 1 mpa_id: 15270 wdpa_id: 555567687
416 2016-04-20 14:21:00.252236 count: 3 mpa_id: 4091 wdpa_id: 306329
417 2014-11-05 16:39:50.501722 count: 1 mpa_id: 4063 wdpa_id: 306292
418 2014-11-05 16:39:50.619885 count: 1 mpa_id: 4068 wdpa_id: 30629

530 2016-04-01 14:21:52.020695 count: 1 mpa_id: 2077 wdpa_id: 65398
531 2016-03-29 12:41:38.255006 count: 2 mpa_id: 5742 wdpa_id: 555516398
532 2014-11-05 16:23:16.728411 count: 1 mpa_id: 12455 wdpa_id: 555515584
533 2014-11-05 16:23:16.786148 count: 1 mpa_id: 12456 wdpa_id: 555515587
534 2018-01-22 09:20:39.917788 count: 11 mpa_id: 5745 wdpa_id: 555516418
535 2014-11-05 16:41:36.448301 count: 1 mpa_id: 5670 wdpa_id: 555515592
536 2017-11-10 14:00:35.489717 count: 9 mpa_id: 5434 wdpa_id: 555512149
537 2016-03-29 12:26:15.531103 count: 3 mpa_id: 9812 wdpa_id: 67347
538 2016-03-29 12:17:01.116768 count: 2 mpa_id: 12449 wdpa_id: 555515566
539 2016-04-01 14:21:46.895541 count: 4 mpa_id: 2638 wdpa_id: 101680
540 2016-03-29 12:41:42.944867 count: 2 mpa_id: 2104 wdpa_id: 67059
541 2016-04-01 14:24:13.216300 count: 1 mpa_id: 2076 wdpa_id: 65391
542 2016-03-29 12:55:53.711964 count: 2 mpa_id: 11173 wdpa_id: 305170
543 2016-03-29 12:34:00.525737 count: 2 mpa_id: 2094 wdpa_id: 67041
544 2016-04-2

668 2016-03-29 12:46:57.102873 count: 2 mpa_id: 2635 wdpa_id: 101669
669 2016-03-29 12:47:28.766114 count: 2 mpa_id: 2602 wdpa_id: 99836
670 2016-03-29 12:33:07.622733 count: 2 mpa_id: 2640 wdpa_id: 101688
671 2016-03-29 12:45:40.209869 count: 2 mpa_id: 2642 wdpa_id: 101709
672 2016-03-29 12:47:28.768354 count: 2 mpa_id: 2643 wdpa_id: 101713
673 2016-03-29 12:52:45.551059 count: 2 mpa_id: 2600 wdpa_id: 99829
674 2014-11-05 16:38:08.084295 count: 1 mpa_id: 2644 wdpa_id: 101721
675 2016-03-29 12:26:16.253416 count: 3 mpa_id: 2093 wdpa_id: 67038
676 2016-03-29 12:34:41.361354 count: 2 mpa_id: 2096 wdpa_id: 67043
677 2017-11-16 10:07:58.803370 count: 3 mpa_id: 2139 wdpa_id: 67406
678 2016-03-29 12:38:59.816088 count: 2 mpa_id: 2593 wdpa_id: 99743
679 2016-03-29 12:44:44.715477 count: 2 mpa_id: 2073 wdpa_id: 65371
680 2016-03-29 12:35:03.641436 count: 2 mpa_id: 2595 wdpa_id: 99748
681 2016-03-29 12:35:25.382239 count: 2 mpa_id: 2597 wdpa_id: 99751
682 2016-03-29 12:35:57.639156 count: 2 mpa

791 2014-11-05 16:37:29.164916 count: 1 mpa_id: 2183 wdpa_id: 67851
792 2014-11-05 16:37:29.062660 count: 1 mpa_id: 2180 wdpa_id: 67845
793 2016-04-01 14:22:26.433473 count: 1 mpa_id: 4299 wdpa_id: 308496
794 2016-04-20 14:19:39.122267 count: 2 mpa_id: 15236 wdpa_id: 555567611
795 2014-11-05 16:22:15.485116 count: 1 mpa_id: 11196 wdpa_id: 308337
796 2016-04-20 14:19:45.559394 count: 2 mpa_id: 4086 wdpa_id: 306322
797 2014-11-05 16:27:23.982347 count: 1 mpa_id: 15211 wdpa_id: 555567565
798 2016-04-01 14:23:03.198970 count: 1 mpa_id: 1570 wdpa_id: 23095
799 2014-11-05 16:21:28.404808 count: 1 mpa_id: 9810 wdpa_id: 67310
800 2016-04-20 14:19:50.989622 count: 2 mpa_id: 4087 wdpa_id: 306323
801 2016-04-20 14:19:57.208858 count: 2 mpa_id: 4085 wdpa_id: 306321
802 2014-11-05 16:21:56.149424 count: 1 mpa_id: 10668 wdpa_id: 167344
803 2016-04-20 14:20:02.980884 count: 2 mpa_id: 11191 wdpa_id: 308329
804 2014-11-05 16:37:24.497402 count: 1 mpa_id: 2085 wdpa_id: 66651
805 2014-11-05 16:37:28.6142

929 2014-11-05 16:23:19.244877 count: 1 mpa_id: 12461 wdpa_id: 555516236
930 2016-04-20 14:40:01.341985 count: 2 mpa_id: 15179 wdpa_id: 555567023
931 2014-11-05 16:22:16.703099 count: 1 mpa_id: 11232 wdpa_id: 308430
932 2016-04-20 14:40:06.873067 count: 2 mpa_id: 15242 wdpa_id: 555567625
933 2016-04-20 14:40:13.482775 count: 2 mpa_id: 2130 wdpa_id: 67301
934 2014-11-05 16:22:16.843118 count: 1 mpa_id: 11236 wdpa_id: 308450
935 2016-04-20 14:40:17.536526 count: 2 mpa_id: 9813 wdpa_id: 67353
936 2016-04-20 14:40:22.631648 count: 2 mpa_id: 15215 wdpa_id: 555567572
937 2016-03-29 12:03:50.843398 count: 2 mpa_id: 11239 wdpa_id: 308506
938 2014-11-05 16:37:26.186928 count: 1 mpa_id: 2112 wdpa_id: 67268
939 2014-11-05 16:37:26.342703 count: 1 mpa_id: 2116 wdpa_id: 67279
940 2016-04-20 14:40:33.900612 count: 2 mpa_id: 15228 wdpa_id: 555567599
941 2016-04-20 14:40:39.047898 count: 2 mpa_id: 15255 wdpa_id: 555567649
942 2016-04-20 14:40:43.652904 count: 2 mpa_id: 15244 wdpa_id: 555567632
943 201

In [13]:
from importlib import reload
from wdpa import merge
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [49]:
remove_log = '/home/mpatlas/workspace/log/removewdpa_CAN_log.json'
wdpa2remove = merge.getRemoveWdpaList(logfile=remove_log)

In [50]:
len(wdpa2remove)

48

In [51]:
rm_mpas = Mpa.objects.filter(wdpa_id__in=wdpa2remove).order_by('country')
len(rm_mpas)

48

In [52]:
for m in rm_mpas:
    print(m.mpa_id, m.wdpa_id, m.country, m.sovereign, m.is_mpa, m.no_take, m.no_take_area, m.name, m.designation_eng)

11182 308270 CAN CAN False Not Reported 0.0 Aire de concentration d'oiseaux aquatiques de Pointe à Georges-Morneau, Pointe du Moulin Water fowl gathering area
9808 67305 CAN CAN False Not Reported 0.0 Saint-Roch-des-Aulnaies Water fowl gathering area
9750 18154 CAN CAN False Not Reported 0.0 Esquimalt Lagoon Migratory Bird Sanctuary
11216 308386 CAN CAN False Not Reported 0.0 Aire de concentration d'oiseaux aquatiques de la pointe à Jim Hearst - Pointe au Tonnerre Water fowl gathering area
421 4154 CAN CAN True All 0.6131 Îles de l'Estuaire National Wildlife Area
2084 66628 CAN CAN True None 0.0 Saguenay - Saint-Laurent Marine Park
7700529 5073 CAN CAN False Not Reported None Betchouane Migratory Bird Sanctuary
7700530 5074 CAN CAN False None None Baie des Loups Migratory Bird Sanctuary
7700424 4161 CAN CAN False None None Brador Bay Migratory Bird Sanctuary
7700820 10448 CAN CAN False None None Île à la Brume Migratory Bird Sanctuary
7700531 5076 CAN CAN False None None Cap-Saint-Igna

In [53]:
merge.removeMpasByWdpaId(wdpa2remove)

In [54]:
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [14]:
wdpa2add = merge.getAddWdpaList()

In [15]:
len(wdpa2add)

101

In [16]:
wdpapid2add = merge.getAddWdpaPidList(verbose=True)

18638_B : 1 processed
555516378_B : 2 processed
555516381_B : 3 processed
555516353_B : 4 processed
555515563_B : 5 processed
65755_B : 6 processed
65371_B : 7 processed
555516361_B : 8 processed
613_B : 9 processed
555515590_B : 10 processed
13109_B : 11 processed
555621601_B : 12 processed
65090_B : 13 processed
101688_B : 14 processed
555516314_B : 15 processed
555516200_B : 16 processed
13394_B : 17 processed
303295_B : 18 processed
555516373_B : 19 processed
555557363_B : 20 processed
555516429_B : 21 processed
66594_B : 22 processed
555558201_B : 23 processed
101667_B : 24 processed
555516213_B : 25 processed
555516382_B : 26 processed
555515580_B : 27 processed
555515666_B : 28 processed
555515529_B : 29 processed
67356_B : 30 processed
101721_B : 31 processed
101646_B : 32 processed
67051_B : 33 processed
18424_B : 34 processed
18422_B : 35 processed
555515574_B : 36 processed
555515531_B : 37 processed
101598_B : 38 processed
555516400_B : 39 processed
67043_B : 40 processed
1

In [17]:
len(wdpapid2add)

287

In [16]:
# Assign records with zones to be the first zone.  Do not rerun.
# In separate notebook

In [18]:
allpid2add = wdpapid2add + [str(int(i)) for i in wdpa2add]
len(allpid2add)

388

In [19]:
reload(merge)
addpoint_geolog = '/home/mpatlas/workspace/log/addpoint_CAN_log.json'
addpoly_log = '/home/mpatlas/workspace/log/addpoly_CAN_log.json'
addpoly_geolog = '/home/mpatlas/workspace/log/addpoly_CAN_geo_log.json'

In [61]:
points = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=points, poly=False, logfile=None, geologfile=addpoint_geolog, dryrun=True)

True

In [62]:
# post_save.disconnect(mpa_post_save, sender=Mpa)
polys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=polys, poly=True, logfile=addpoly_log, geologfile=addpoly_geolog, dryrun=True)
# post_save.connect(mpa_post_save, sender=Mpa)

1/355 adding/updating wdpa_pid 100660_B with mpa_id None
2/355 adding/updating wdpa_pid 100672_B with mpa_id None
3/355 adding/updating wdpa_pid 100676_B with mpa_id None
4/355 adding/updating wdpa_pid 101417_B with mpa_id None
5/355 adding/updating wdpa_pid 101421_B with mpa_id None
6/355 adding/updating wdpa_pid 101534_B with mpa_id None
7/355 adding/updating wdpa_pid 101597_B with mpa_id None
8/355 adding/updating wdpa_pid 101598_B with mpa_id None
9/355 adding/updating wdpa_pid 101635_B with mpa_id None
10/355 adding/updating wdpa_pid 101640_B with mpa_id None
11/355 adding/updating wdpa_pid 101645_B with mpa_id None
12/355 adding/updating wdpa_pid 101646_B with mpa_id None
13/355 adding/updating wdpa_pid 101651_B with mpa_id None
14/355 adding/updating wdpa_pid 101659_B with mpa_id None
15/355 adding/updating wdpa_pid 101666_B with mpa_id None
16/355 adding/updating wdpa_pid 101667_B with mpa_id None
17/355 adding/updating wdpa_pid 101669_B with mpa_id None
18/355 adding/updating 

143/355 adding/updating wdpa_pid 555515611_B with mpa_id None
144/355 adding/updating wdpa_pid 555515625_B with mpa_id None
145/355 adding/updating wdpa_pid 555515628_B with mpa_id None
146/355 adding/updating wdpa_pid 555515666_B with mpa_id None
147/355 adding/updating wdpa_pid 555516156_B with mpa_id None
148/355 adding/updating wdpa_pid 555516157_B with mpa_id None
149/355 adding/updating wdpa_pid 555516158_B with mpa_id None
150/355 adding/updating wdpa_pid 555516185_B with mpa_id None
151/355 adding/updating wdpa_pid 555516192_B with mpa_id None
152/355 adding/updating wdpa_pid 555516197_B with mpa_id None
153/355 adding/updating wdpa_pid 555516200_B with mpa_id None
154/355 adding/updating wdpa_pid 555516202_B with mpa_id None
155/355 adding/updating wdpa_pid 555516212_B with mpa_id None
156/355 adding/updating wdpa_pid 555516213_B with mpa_id None
157/355 adding/updating wdpa_pid 555516215_B with mpa_id None
158/355 adding/updating wdpa_pid 555516223_B with mpa_id None
159/355 

277/355 adding/updating wdpa_pid 555593896 with mpa_id None
278/355 adding/updating wdpa_pid 555593900 with mpa_id None
279/355 adding/updating wdpa_pid 555593901 with mpa_id None
280/355 adding/updating wdpa_pid 555621544 with mpa_id None
281/355 adding/updating wdpa_pid 555621551_B with mpa_id None
282/355 adding/updating wdpa_pid 555621553_B with mpa_id None
283/355 adding/updating wdpa_pid 555621557_B with mpa_id None
284/355 adding/updating wdpa_pid 555621601_B with mpa_id None
285/355 adding/updating wdpa_pid 555621668 with mpa_id None
286/355 adding/updating wdpa_pid 555621669 with mpa_id None
287/355 adding/updating wdpa_pid 555621670 with mpa_id None
288/355 adding/updating wdpa_pid 555621671 with mpa_id None
289/355 adding/updating wdpa_pid 555621675_B with mpa_id None
290/355 adding/updating wdpa_pid 555621682 with mpa_id None
291/355 adding/updating wdpa_pid 613_B with mpa_id None
292/355 adding/updating wdpa_pid 628_B with mpa_id None
293/355 adding/updating wdpa_pid 632_B

True

In [20]:
reload(merge)
updatepoint_geolog = '/home/mpatlas/workspace/log/updatepoint_CAN_log.json'
updatepoly_log = '/home/mpatlas/workspace/log/updatepoly_CAN_log.json'
updatepoly_geolog = '/home/mpatlas/workspace/log/updatepoly_CAN_geo_log.json'

In [21]:
wdpa_filter = (
    Q(iso3__icontains='CAN') | Q(parent_iso3__icontains='CAN')
)
polys = Wdpa2018Poly.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
points = Wdpa2018Point.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
allpid2update = list(set(list(polys) + list(points)))

polys_nonmarine = Wdpa2018Poly.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('0')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
points_nonmarine = Wdpa2018Point.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('0')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
allpid2update_nonmarine = list(set(list(polys_nonmarine) + list(points_nonmarine)))

allpid2update_nonmarine_inmpatlas = mpaset.filter(wdpa_pid__in=allpid2update_nonmarine).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)

allpid2update_marine012 = allpid2update + list(allpid2update_nonmarine_inmpatlas)

In [22]:
print('marine:', len(allpid2update), 'nonmarine:', len(allpid2update_nonmarine), 'all:', len(allpid2update_marine012))
print('POINTS --', 
      'marine:', Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2update).count(),
      'nonmarine:', Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2update_nonmarine).count()
)
print('POLYGONS --', 
      'marine:', Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2update).count(),
      'nonmarine:', Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2update_nonmarine).count(),
)
print('MPATLAS Records --', 
      'marine:', mpaset.filter(wdpa_pid__in=allpid2update).count(),
      'nonmarine:', mpaset.filter(wdpa_pid__in=allpid2update_nonmarine).count(),
)

marine: 388 nonmarine: 7192 all: 682
POINTS -- marine: 5 nonmarine: 15
POLYGONS -- marine: 383 nonmarine: 7177
MPATLAS Records -- marine: 385 nonmarine: 294


In [23]:
# All marine=1,2 sites and only marine=0 that exist in MPAtlas
updatepoints = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2update_marine012).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=updatepoints, poly=False, logfile=None, geologfile=updatepoint_geolog, dryrun=True)

1/5 adding/updating wdpa_pid 198337 with mpa_id 6896
ERROR processing mpa 6896:  'NoneType' object has no attribute 'hexewkb'
2/5 adding/updating wdpa_pid 220257 with mpa_id 6974
ERROR processing mpa 6974:  'NoneType' object has no attribute 'hexewkb'
3/5 adding/updating wdpa_pid 555547533 with mpa_id 9500
ERROR processing mpa 9500:  'NoneType' object has no attribute 'hexewkb'
4/5 adding/updating wdpa_pid 67855 with mpa_id 6710
ERROR processing mpa 6710:  'NoneType' object has no attribute 'hexewkb'
5/5 adding/updating wdpa_pid 902536 with mpa_id 9482
ERROR processing mpa 9482:  'NoneType' object has no attribute 'hexewkb'


True

In [24]:
# All marine=1,2 sites and only marine=0 that exist in MPAtlas
updatepolys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2update_marine012).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=updatepolys, poly=True, logfile=updatepoly_log, geologfile=updatepoly_geolog, dryrun=True)

1/677 adding/updating wdpa_pid 100660_A with mpa_id 2606
2/677 adding/updating wdpa_pid 100672_A with mpa_id 2607
    PID: 100672_A has 1 field diffs  and GEOM diff
3/677 adding/updating wdpa_pid 100676_A with mpa_id 2608
4/677 adding/updating wdpa_pid 101417_A with mpa_id 2619
    PID: 101417_A has 1 field diffs  and GEOM diff
5/677 adding/updating wdpa_pid 101421_A with mpa_id 2620
    PID: 101421_A has 1 field diffs  and GEOM diff
6/677 adding/updating wdpa_pid 101534_A with mpa_id 2623
    PID: 101534_A has 1 field diffs  and GEOM diff
7/677 adding/updating wdpa_pid 101586 with mpa_id 2624
8/677 adding/updating wdpa_pid 101597_A with mpa_id 2625
    PID: 101597_A has 1 field diffs  and GEOM diff
9/677 adding/updating wdpa_pid 101598_A with mpa_id 2626
    PID: 101598_A has 1 field diffs  and GEOM diff
10/677 adding/updating wdpa_pid 101635_A with mpa_id 2627
    PID: 101635_A has 1 field diffs  and GEOM diff
11/677 adding/updating wdpa_pid 101640_A with mpa_id 2628
12/677 adding/up

84/677 adding/updating wdpa_pid 18426_A with mpa_id 1388
    PID: 18426_A has 7 field diffs  and GEOM diff
85/677 adding/updating wdpa_pid 18432_A with mpa_id 1389
    PID: 18432_A has 8 field diffs  and GEOM diff
86/677 adding/updating wdpa_pid 18499_A with mpa_id 1390
    PID: 18499_A has 9 field diffs  and GEOM diff
87/677 adding/updating wdpa_pid 18502 with mpa_id 1391
    PID: 18502 has 10 field diffs  and GEOM diff
88/677 adding/updating wdpa_pid 18503_A with mpa_id 1392
    PID: 18503_A has 8 field diffs  and GEOM diff
89/677 adding/updating wdpa_pid 18504_A with mpa_id 1393
    PID: 18504_A has 9 field diffs  and GEOM diff
90/677 adding/updating wdpa_pid 18506 with mpa_id 1394
    PID: 18506 has 11 field diffs  and GEOM diff
91/677 adding/updating wdpa_pid 18638_A with mpa_id 1400
    PID: 18638_A has 8 field diffs  and GEOM diff
92/677 adding/updating wdpa_pid 18640_A with mpa_id 9754
    PID: 18640_A has 9 field diffs  and GEOM diff
93/677 adding/updating wdpa_pid 18672 with 

161/677 adding/updating wdpa_pid 306368 with mpa_id 4115
    PID: 306368 has 3 field diffs  and GEOM diff
162/677 adding/updating wdpa_pid 306369 with mpa_id 4116
    PID: 306369 has 4 field diffs  and GEOM diff
163/677 adding/updating wdpa_pid 306372 with mpa_id 4119
    PID: 306372 has 3 field diffs  and GEOM diff
164/677 adding/updating wdpa_pid 306373 with mpa_id 4120
    PID: 306373 has 3 field diffs  and GEOM diff
165/677 adding/updating wdpa_pid 306374 with mpa_id 4121
    PID: 306374 has 3 field diffs  and GEOM diff
166/677 adding/updating wdpa_pid 306375 with mpa_id 4122
    PID: 306375 has 3 field diffs  and GEOM diff
167/677 adding/updating wdpa_pid 306376 with mpa_id 4123
    PID: 306376 has 3 field diffs  and GEOM diff
168/677 adding/updating wdpa_pid 306380 with mpa_id 4125
    PID: 306380 has 3 field diffs  and GEOM diff
169/677 adding/updating wdpa_pid 306381 with mpa_id 4126
    PID: 306381 has 9 field diffs  and GEOM diff
170/677 adding/updating wdpa_pid 306382 with m

239/677 adding/updating wdpa_pid 308321 with mpa_id 4266
    PID: 308321 has 2 field diffs  and GEOM diff
240/677 adding/updating wdpa_pid 308323 with mpa_id 4267
    PID: 308323 has 2 field diffs  and GEOM diff
241/677 adding/updating wdpa_pid 308324 with mpa_id 11190
    PID: 308324 has 3 field diffs  and GEOM diff
242/677 adding/updating wdpa_pid 308328 with mpa_id 4269
    PID: 308328 has 3 field diffs  and GEOM diff
243/677 adding/updating wdpa_pid 308329 with mpa_id 11191
    PID: 308329 has 3 field diffs  and GEOM diff
244/677 adding/updating wdpa_pid 308330 with mpa_id 4270
    PID: 308330 has 3 field diffs  and GEOM diff
245/677 adding/updating wdpa_pid 308331 with mpa_id 4271
    PID: 308331 has 3 field diffs  and GEOM diff
246/677 adding/updating wdpa_pid 308332 with mpa_id 11192
    PID: 308332 has 3 field diffs  and GEOM diff
247/677 adding/updating wdpa_pid 308334 with mpa_id 11193
    PID: 308334 has 3 field diffs  and GEOM diff
248/677 adding/updating wdpa_pid 308335 wi

316/677 adding/updating wdpa_pid 308496 with mpa_id 4299
    PID: 308496 has 12 field diffs  and GEOM diff
317/677 adding/updating wdpa_pid 308498 with mpa_id 4300
    PID: 308498 has 13 field diffs  and GEOM diff
318/677 adding/updating wdpa_pid 308499 with mpa_id 4301
    PID: 308499 has 13 field diffs  and GEOM diff
319/677 adding/updating wdpa_pid 308500 with mpa_id 4302
    PID: 308500 has 11 field diffs  and GEOM diff
320/677 adding/updating wdpa_pid 308505 with mpa_id 11238
    PID: 308505 has 5 field diffs  and GEOM diff
321/677 adding/updating wdpa_pid 308506 with mpa_id 11239
    PID: 308506 has 6 field diffs  and GEOM diff
322/677 adding/updating wdpa_pid 308518 with mpa_id 11242
    PID: 308518 has 5 field diffs  and GEOM diff
323/677 adding/updating wdpa_pid 308699 with mpa_id 4338
    PID: 308699 has 9 field diffs  and GEOM diff
324/677 adding/updating wdpa_pid 39501_A with mpa_id 1889
    PID: 39501_A has 10 field diffs  and GEOM diff
325/677 adding/updating wdpa_pid 414

389/677 adding/updating wdpa_pid 555516319_A with mpa_id 5694
    PID: 555516319_A has 9 field diffs  and GEOM diff
390/677 adding/updating wdpa_pid 555516321 with mpa_id 5695
    PID: 555516321 has 9 field diffs  and GEOM diff
391/677 adding/updating wdpa_pid 555516352_A with mpa_id 5696
    PID: 555516352_A has 7 field diffs  and GEOM diff
392/677 adding/updating wdpa_pid 555516353_A with mpa_id 5697
    PID: 555516353_A has 5 field diffs  and GEOM diff
393/677 adding/updating wdpa_pid 555516354_A with mpa_id 5698
    PID: 555516354_A has 7 field diffs  and GEOM diff
394/677 adding/updating wdpa_pid 555516355_A with mpa_id 5699
    PID: 555516355_A has 9 field diffs  and GEOM diff
395/677 adding/updating wdpa_pid 555516356_A with mpa_id 5700
    PID: 555516356_A has 6 field diffs  and GEOM diff
396/677 adding/updating wdpa_pid 555516357_A with mpa_id 5701
    PID: 555516357_A has 6 field diffs  and GEOM diff
397/677 adding/updating wdpa_pid 555516358_A with mpa_id 5702
    PID: 55551

461/677 adding/updating wdpa_pid 555567022 with mpa_id 15178
    PID: 555567022 has 3 field diffs  and GEOM diff
462/677 adding/updating wdpa_pid 555567023 with mpa_id 15179
    PID: 555567023 has 3 field diffs  and GEOM diff
463/677 adding/updating wdpa_pid 555567025 with mpa_id 15180
    PID: 555567025 has 3 field diffs  and GEOM diff
464/677 adding/updating wdpa_pid 555567027 with mpa_id 15181
    PID: 555567027 has 2 field diffs  and GEOM diff
465/677 adding/updating wdpa_pid 555567028 with mpa_id 15182
    PID: 555567028 has 3 field diffs  and GEOM diff
466/677 adding/updating wdpa_pid 555567029 with mpa_id 15183
    PID: 555567029 has 2 field diffs  and GEOM diff
467/677 adding/updating wdpa_pid 555567032 with mpa_id 15184
    PID: 555567032 has 3 field diffs  and GEOM diff
468/677 adding/updating wdpa_pid 555567035 with mpa_id 15185
    PID: 555567035 has 3 field diffs  and GEOM diff
469/677 adding/updating wdpa_pid 555567037 with mpa_id 15186
    PID: 555567037 has 2 field diff

534/677 adding/updating wdpa_pid 555567643 with mpa_id 15252
    PID: 555567643 has 2 field diffs  and GEOM diff
535/677 adding/updating wdpa_pid 555567644 with mpa_id 15253
    PID: 555567644 has 3 field diffs  and GEOM diff
536/677 adding/updating wdpa_pid 555567646 with mpa_id 15254
    PID: 555567646 has 2 field diffs  and GEOM diff
537/677 adding/updating wdpa_pid 555567649 with mpa_id 15255
    PID: 555567649 has 3 field diffs  and GEOM diff
538/677 adding/updating wdpa_pid 555567654 with mpa_id 15256
    PID: 555567654 has 3 field diffs  and GEOM diff
539/677 adding/updating wdpa_pid 555567657 with mpa_id 15257
    PID: 555567657 has 2 field diffs  and GEOM diff
540/677 adding/updating wdpa_pid 555567659 with mpa_id 15258
    PID: 555567659 has 2 field diffs  and GEOM diff
541/677 adding/updating wdpa_pid 555567660 with mpa_id 15259
    PID: 555567660 has 3 field diffs  and GEOM diff
542/677 adding/updating wdpa_pid 555567662 with mpa_id 15260
    PID: 555567662 has 3 field diff

610/677 adding/updating wdpa_pid 67120 with mpa_id 2107
    PID: 67120 has 4 field diffs  and GEOM diff
611/677 adding/updating wdpa_pid 67141 with mpa_id 9797
    PID: 67141 has 5 field diffs  and GEOM diff
612/677 adding/updating wdpa_pid 67152_A with mpa_id 2108
    PID: 67152_A has 7 field diffs  and GEOM diff
613/677 adding/updating wdpa_pid 67197 with mpa_id 2109
    PID: 67197 has 12 field diffs  and GEOM diff
614/677 adding/updating wdpa_pid 67208 with mpa_id 2110
    PID: 67208 has 3 field diffs  and GEOM diff
615/677 adding/updating wdpa_pid 67264 with mpa_id 2111
    PID: 67264 has 3 field diffs  and GEOM diff
616/677 adding/updating wdpa_pid 67268 with mpa_id 2112
    PID: 67268 has 2 field diffs  and GEOM diff
617/677 adding/updating wdpa_pid 67269 with mpa_id 9799
    PID: 67269 has 2 field diffs  and GEOM diff
618/677 adding/updating wdpa_pid 67271 with mpa_id 2113
    PID: 67271 has 2 field diffs  and GEOM diff
619/677 adding/updating wdpa_pid 67272 with mpa_id 9801
   

True

In [25]:
mpaset.filter(wdpa_pid__isnull=False).update(
    no_take_wdpa=F('no_take'), no_take_area_wdpa=F('no_take_area')
)

1037

In [26]:
nt = ('All', 'Part', 'None')
ntupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    (~Q(no_take_wdpa__in=('All')) & Q(no_take_mpatlas__in=('All')))
    | (~Q(no_take_wdpa__in=('All', 'Part')) & Q(no_take_mpatlas__in=('All', 'Part')))
    | (Q(no_take_wdpa__in=('Not Applicable')) & Q(no_take_mpatlas__in=('All', 'Part', 'None')))
)
ntupdate.update(no_take=F('no_take_mpatlas'))

5

In [27]:
ntareaupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    Q(no_take_area_mpatlas__isnull=False) &
    (
        Q(no_take_area_wdpa__isnull=True)
        | Q(no_take_area_mpatlas__gt=F('no_take_area_wdpa'))
    )
)
ntareaupdate.update(no_take_area=F('no_take_area_mpatlas'))

5

In [28]:
ntchanges = ntupdate | ntareaupdate
for n in ntchanges:
    print(n.mpa_id, n.wdpa_id, n.wdpa_pid, n.name, n.designation, n.designation_eng, n.no_take_wdpa, n.no_take_mpatlas, n.no_take_area_wdpa, n.no_take_area_mpatlas)

5431 555512142 555512142 Eastport – Duck Island Marine Protected Area Marine Protected Area Marine Protected Area Not Reported All 0.0 2.1
1362 18161 18161_A Alaksen National Wildlife Area National Wildlife Area National Wildlife Area Not Applicable All 0.0 0.51
3876 305214 305214 Gully Marine Protected Area Marine Protected Area Marine Protected Area Not Reported Part 0.0 475.0
5432 555512144 555512144 Basin Head Marine Protected Area Marine Protected Area Marine Protected Area Not Reported Part 0.0 0.24
5435 555512150 555512150 Gwaii Haanas National Marine Conservation Area Reserve & Haida Heritage Site National Marine Conservation Area National Marine Conservation Area Not Reported Part 0.0 102.0


In [29]:
ntchanges = ntupdate | ntareaupdate

import csv

with open('/home/mpatlas/workspace/log/notakechange_CAN_log.csv', 'w', newline='') as csvfile:
    fieldnames = ['mpa_id', 'wdpa_id', 'wdpa_pid', 'name', 'designation', 'designation_eng', 'no_take_wdpa', 'no_take_mpatlas', 'no_take_area_wdpa', 'no_take_area_mpatlas']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in ntchanges.values(*fieldnames):
        writer.writerow(row)

In [31]:
for n in ntchanges:
    try:
        w2018 = Wdpa2018Poly.objects.get(wdpa_pid=n.wdpa_pid)
    except:
        w2018 = Wdpa2018Point.objects.get(wdpa_pid=n.wdpa_pid)
    try:
        w2014 = Wdpa2014Polygon.objects.filter(wdpa_pid=F('wdpa_pid'))[0]
    except:
        try:
            w2014 = Wdpa2014Point.objects.filter(wdpa_pid=F('wdpa_pid'))[0]
        except:
            try:
                w2014 = Wdpa2014Polygon.objects.filter(wdpa_id=F('wdpa_id'))[0]
            except:
                w2014 = Wdpa2014Point.objects.filter(wdpa_id=F('wdpa_id'))[0]
    if (n.no_take == w2014.no_take) or (n.no_take_area == w2014.no_tk_area):
        print(w2018.name, w2018.desig, n.no_take_mpatlas, w2018.no_take, w2014.no_take, n.no_take_area_mpatlas, w2018.no_tk_area, w2014.no_tk_area)

In [32]:
ntchanges2 = ntupdate | ntareaupdate
ntchanges.count()

5

In [33]:
mpaset = mpas_all_nogeom.filter(
    Q(country__icontains='CAN') | Q(sovereign__icontains='CAN')
)

from mpatlas.utils import cartompa
cartompa.purgeCartoMpas()
cartompa.updateAllMpas(mpas=mpaset)

Processing 10 of 1096 mpa records at a time
Records [0 - 9]
Records [10 - 19]
Records [20 - 29]
Records [30 - 39]
Records [40 - 49]
Records [50 - 59]
Records [60 - 69]
ERROR processing mpa 68808462:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68808462
Records [70 - 79]
Records [80 - 89]
Records [90 - 99]
Records [100 - 109]
Records [110 - 119]
Records [120 - 129]
Records [130 - 139]
Records [140 - 149]
Records [150 - 159]
Records [160 - 169]
Records [170 - 179]
Records [180 - 189]
Records [190 - 199]
Records [200 - 209]
Records [210 - 219]
Records [220 - 229]
Records [230 - 239]
Records [240 - 249]
Records [250 - 259]
Records [260 - 269]
Records [270 - 279]
Records [280 - 289]
Records [290 - 299]
Records [300 - 309]
Records [310 - 319]
Records [320 - 329]
Records [330 - 339]
Records [340 - 349]
Records [350 - 359]
Records [360 - 369]
Records [370 - 379]
Records [380 - 389]
Records [390 - 399]
Records [400 - 409]
Records [410 - 419]
Records [420 - 429]
Records [430 - 439]

[68808462, 6896, 6974, 9500, 9482, 6710]

In [53]:
Mpa.objects.count()

16944